<a href="https://colab.research.google.com/github/alvumu/BDGE/blob/main/ClickHouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ClickHouse**

ClickHouse representa un sistema de gestión de bases de datos (DBMS) de código abierto, centrado en columnas y diseñado específicamente para el procesamiento analítico en línea (OLAP). Esta herramienta innovadora fue desarrollada por Yandex y actualmente impulsa la segunda plataforma de análisis web más extensa, Yandex Metrica. Destacando como el primer almacén de datos SQL de código abierto, ClickHouse ha logrado igualar y superar la escalabilidad y rendimiento de bases de datos reconocidas como Veryica y Snowflake. Su enfoque columnar, combinado con una eficiente gestión de datos, lo posiciona como una opción destacada para abordar desafíos analíticos en entornos que demandan respuestas rápidas y procesamiento eficaz de grandes volúmenes de datos.




# Características

Algunas de las características que ofrece este sistema de gestión de bases de datos son los siguientes :  

* Una base de datos orientada a **columna**, sólo almacena los datos.
* Los **datos** se comprimen reduciendo el espacio y se **almacenan** en **disco**
* Capaz de **procesar** en **paralelo** los datos exprimiendo las capacidades multicores de los servidores.
* Destaca por procesar datos mediante vectores, **optimizando** la **eficiencia** de la CPU
* Utiliza el **lenguaje SQL** para ejecución de las sentencias.
* Permite **actualizar** los **datos en tiempo real** sin bloqueo.
* Proporciona **indexación** por **clave primaria**. Hace posible recuperar la información almacenada para un determinado valor o un rango de valores con muy baja latencia y pocos milisegundos.
* Puede trabajar con millones de registros en bruto (sin realizar preprocesamientoo previo de los datos) consiguiendo ejecutar **sentencias sin ninguna latencia**
* Soporte de **replicación** e **integridad** de datos.



# Instalación

A continuación, se procede a la instalación del software de ClickHouse en un entorno Python, con el objetivo de habilitar su uso en Google Colab. Se proporcionarán instrucciones detalladas sobre los pasos necesarios para lograr una integración exitosa de ClickHouse con Python.

In [2]:
!pip install clickhouse-connect
!pip install clickhouse-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.5/964.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.4/740.4 kB 7.8 MB/s eta 0:00:00


In [3]:
import clickhouse_connect

client = clickhouse_connect.get_client(host='fgi402x8e7.europe-west4.gcp.clickhouse.cloud', port=8443, username='default', password='SY_rjdkJKk7oU')

In [7]:
import gzip
from urllib.request import Request,urlopen
import io
import os
import os.path as path

def download_csv(baseurl, filename):
    file = path.abspath(path.join(os.getcwd(),filename))
    request = Request(baseurl + filename + '.gz')
    response = urlopen(request)
    buf = io.BytesIO(response.read())
    f = gzip.GzipFile(fileobj=buf)
    data = f.read()
    with open (filename, 'wb') as ff:
      ff.write(data)

baseurl = 'https://raw.githubusercontent.com/dsevilla/bdge-data/master/es.stackoverflow/'
download_csv(baseurl, 'Posts.csv')
download_csv(baseurl, 'Users.csv')
download_csv(baseurl, 'Tags.csv')
download_csv(baseurl, 'Comments.csv')
download_csv(baseurl, 'Votes.csv')

In [9]:
client.query("CREATE DATABASE IF NOT EXISTS stackoverflow")

In [30]:
client.query("DROP TABLE IF EXISTS stackoverflow.PostsPart")
client.query("""
CREATE TABLE IF NOT EXISTS stackoverflow.PostsPart
(
    Id Int32,
    AcceptedAnswerId Nullable(Int32) DEFAULT NULL,
    AnswerCount Nullable(Int32) DEFAULT 0,
    Body String,
    ClosedDate Nullable(DateTime) DEFAULT NULL,
    CommentCount Nullable(Int32) DEFAULT 0,
    CommunityOwnedDate Nullable(DateTime) DEFAULT NULL,
    CreationDate Nullable(DateTime) DEFAULT NULL,
    FavoriteCount Nullable(Int32) DEFAULT 0,
    LastActivityDate Nullable(DateTime) DEFAULT NULL,
    LastEditDate Nullable(DateTime) DEFAULT NULL,
    LastEditorDisplayName String,
    LastEditorUserId Nullable(Int32) DEFAULT NULL,
    OwnerDisplayName String,
    OwnerUserId Nullable(Int32) DEFAULT NULL,
    ParentId Nullable(Int32) DEFAULT NULL,
    PostTypeId UInt8,
    Score Int32 DEFAULT 0,
    Tags String,
    Title String,
    ViewCount Nullable(Int32) DEFAULT 0
) ENGINE = MergeTree
 PRIMARY KEY Id
 ORDER BY Id;""")

Tras crear la tabla, se importan que completarán la tabla. Para ello se usa la función **insert_file**

In [31]:
from clickhouse_connect.driver.tools import insert_file

insert_file(client, 'PostsPart', 'Posts.csv',
            settings={'input_format_allow_errors_ratio': .2,
                      'input_format_allow_errors_num': 5},
            database = "stackoverflow")


Comprobamos que los datos se han añadido a la tabla correctamente

In [32]:
q = client.query('SELECT count(*) FROM stackoverflow.PostsPart')
q.result_columns

[[76278]]

Se repite el mismo procedimiento para los distintos ficheros


In [33]:
client.query("DROP TABLE IF EXISTS stackoverflow.Users")
client.query("""
CREATE TABLE IF NOT EXISTS stackoverflow.Users
(
    Id Int32,
    AboutMe String,
    AccountID Int32,
    Age Nullable(Int32),
    CreationDate Nullable(DateTime),
    DisplayName String,
    DownVotes Nullable(Int32) DEFAULT 0,
    LastAccessDate Nullable(Datetime),
    Location String,
    ProfileImageUrl String,
    Reputation Nullable(Int32) DEFAULT 0,
    UpVotes Nullable(Int32) DEFAULT 0,
    Views Nullable(Int32) DEFAULT 0,
    WebsiteUrl String
) ENGINE = MergeTree
 PRIMARY KEY Id
 ORDER BY Id;""")

insert_file(client, 'Users', 'Users.csv',
            settings={'input_format_allow_errors_ratio': .2,
                      'input_format_allow_errors_num': 5},
            database = "stackoverflow")


In [34]:

client.query("DROP TABLE IF EXISTS stackoverflow.Tags")
client.query("""
CREATE TABLE IF NOT EXISTS stackoverflow.Tags
(
    Id Int32,
    Count Int32 DEFAULT 0,
    ExcerptPostId Nullable(Int32) DEFAULT NULL,
    TagName String,
    WikiPostId Nullable(Int32) DEFAULT NULL,
) ENGINE = MergeTree()
 PRIMARY KEY Id
 ORDER BY Id;""")

insert_file(client, 'Tags', 'Tags.csv',
            settings={'input_format_allow_errors_ratio': .2,
                      'input_format_allow_errors_num': 5},
            database = "stackoverflow")


In [35]:
client.query("DROP TABLE IF EXISTS stackoverflow.Votes")
client.query("""
CREATE TABLE IF NOT EXISTS stackoverflow.Votes
(
    Id Int32,
    BountyAmount Int32 DEFAULT 0,
    CreationDate Nullable(DateTime) DEFAULT NULL,
    PostId Nullable(Int32) DEFAULT NULL,
    UserId Nullable(Int32) DEFAULT NULL,
    VoteTypeId Int32,
) ENGINE = MergeTree()
 PRIMARY KEY Id
 ORDER BY Id;""")

insert_file(client, 'Votes', 'Votes.csv',
            settings={'input_format_allow_errors_ratio': .2,
                      'input_format_allow_errors_num': 5},
            database = "stackoverflow")

In [36]:
client.query("DROP TABLE IF EXISTS stackoverflow.Comments")
client.query("""
CREATE TABLE IF NOT EXISTS stackoverflow.Comments
(
    Id Int32,
    CreationDate Nullable(DateTime) DEFAULT NULL,
    PostId Nullable(Int32) DEFAULT NULL,
    Score Int32 DEFAULT 0,
    Text String,
    UserDisplayName String,
    UserId Nullable(Int32) DEFAULT NULL

) ENGINE = MergeTree

 PRIMARY KEY Id
 ORDER BY Id;""")

insert_file(client, 'Comments', 'Comments.csv',
            settings={'input_format_allow_errors_ratio': .2,
                      'input_format_allow_errors_num': 5},
            database = "stackoverflow")

In [37]:
q = client.query('SELECT count(*) FROM stackoverflow.Users')
q.result_columns

[[49033]]

In [39]:
q = client.query('SELECT count(*) FROM stackoverflow.Votes')
q.result_columns

[[165621]]

In [40]:
q = client.query('SELECT count(*) FROM stackoverflow.Tags')
q.result_columns

[[1554]]

In [41]:
q = client.query('SELECT count(*) FROM stackoverflow.Comments')
q.result_columns

[[123662]]

Referencias

https://vasexperts.com/es/resources/glossary/clickhouse/ (ClickHouse)

https://www.adictosaltrabajo.com/2019/02/18/introduccion-a-clickhouse/#01 (Caracteristicas)

https://clickhouse.com/docs/en/integrations/python (Instalación)


https://www.percona.com/blog/clickhouse-new-opensource-columnar-database/

https://www.linkedin.com/pulse/clickhouse-un-motor-de-base-sql-super-rápid-ezequiel-paolillo/?originalSubdomain=es
